In [9]:
import os
import ast
from dotenv import load_dotenv
import openai
from openai import OpenAI
import time
from tavily import TavilyClient

load_dotenv()

openai.api_key = os.environ.get('OPENAI_API_KEY')
tavily_api_key = os.environ.get('TAVILY_API_KEY')

In [10]:
def trending_module_summary_from_web(domain):
    query = 'Trending topics in ' + domain
    tavily_client = TavilyClient(api_key=tavily_api_key)
    start = time.time()
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=4000)
    print(f"Time required to search for trending topics: {time.time()-start}sec.")
    
    module_generation_prompt = """As an educational assistant, your goal is to craft 4-6 \
  educational module names and brief summaries based on the search results. \
  Ensure the module names are relevant and provide a concise summary for each. \
  Format the output in JSON, with each key representing a complete module name and its corresponding value being the brief summary.

Search Results: {search_result}

Follow the provided JSON format diligently, incorporating information from the search results into the summaries.
"""
    start = time.time()
    client = OpenAI()
    completion = client.chat.completions.create(
            model = 'gpt-3.5-turbo-1106',
            messages = [
                {'role':'user', 'content': module_generation_prompt.format(search_result = search_result)},
            ],
            response_format = {'type':'json_object'},
            seed = 42,
)
    print(f"Time required to generate module for trending topics: {time.time()-start}sec.")
    output = ast.literal_eval(completion.choices[0].message.content)
    
    return output

In [12]:
from pprint import pprint

domain = "Generative AI"

websearch_response = trending_module_summary_from_web(domain)
print("websearch response:")
pprint(websearch_response,sort_dicts=False)

Time required to search for trending topics: 5.95522403717041sec.
Time required to generate module for trending topics: 5.427016735076904sec.
websearch response:
{'Generative AI Trends for 2024': 'Explore the direction of new generative AI '
                                  'models expected in 2024 based on the '
                                  'explosive breakout trend of 2023, including '
                                  'its potential impact on various industries '
                                  'and its transformative capabilities.',
 'State of AI in 2023 and the Future of Generative AI': 'Learn about the '
                                                        'explosive growth and '
                                                        'new capabilities of '
                                                        'generative AI as '
                                                        'confirmed by the '
                                                        'latest